In [ ]:
import os
import numpy as np
import pandas as pd

import riskflow_jupyter

In [ ]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [ ]:
from riskflow_widgets import Table

In [ ]:
d=Table(description='', settings=to_json({
    'width':600, 'height':300, 'contextMenu':True, 'startRows':3, 'startCols':3, 'minSpareRows':1, 'minSpareCols':1}
))
d

In [ ]:
d.value

In [ ]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))


In [ ]:
import ipywidgets as widgets


In [ ]:
# factor = riskflow_jupyter.rf.utils.Factor('FXVol',('GBP','ZAR'))
factor = riskflow_jupyter.rf.utils.Factor('InterestYieldVol',('CAD_SMILE_ICE',))

vol = riskflow_jupyter.rf.riskfactors.construct_factor( factor,
    cx.current_cfg.params['Price Factors'],
    cx.current_cfg.params['Price Factor Interpolation']
)

In [ ]:
import k3d
import json


def to_json(string):
    """converts a string to json - skips the whitespace for a smaller output"""
    return json.dumps(string, separators=(',', ':'))


def load_table_from_vol(vol):
    '''
    :param vol: a riskfactor representing an array of 2d or 3d surfaces in the
                'param' dict (usually stored as the 'Surface' key)
    :return: a list of lists representing the columns and rows (expiry and moneyness) where
            the vols are in the right cell.
    '''

    def make_table(array, index1=0, index2=1, index3=2):
        sparse_matrix = {}
        for k in array:
            sparse_matrix.setdefault(k[index1], {}).setdefault(k[index2], k[index3])

        df = pd.DataFrame(sparse_matrix).sort_index(level=[0, 1])

        return np.vstack([
            [0.0] + df.columns.values.round(5).tolist(),
            np.hstack(
                [df.index.values.round(5).reshape(-1, 1).tolist(),
                 df.round(5).replace({np.nan: None})]
            )]
        ).tolist()

    if vol.__class__.__name__ in riskflow_jupyter.rf.utils.TwoDimensionalFactors:
        return make_table(vol.param['Surface'].array)
    elif vol.__class__.__name__ in riskflow_jupyter.rf.utils.ThreeDimensionalFactors:
        vol_space = {}
        for t in vol.get_tenor():
            surface = vol.param['Surface'].array
            vol_space.setdefault(
                t, make_table(
                    surface[surface[:, vol.TENOR_INDEX] == t],
                    index1=vol.MONEYNESS_INDEX, index2=vol.EXPIRY_INDEX, index3=3)
            )
        return vol_space


class Three(widgets.HBox):
    def __init__(self, description):
        self.description = widgets.Label(value=description)        
        self.plot = k3d.plot(
            axes=['log(moneyness)', 'expiry','vol(\%)'],
            #menu_visibility=False,
            camera_rotate_speed=3.0
            )

        self.mesh = None
        self.plot_label = None
        self.data = Table(description='', settings=to_json({
            'width':600, 'height':300, 'contextMenu':True, 'minSpareRows':1, 'minSpareCols':1
        }))
        
        self.add_button = widgets.Button(
            description='Add Tenor', tooltip='Add a new vol surface for a new tenor')
        self.del_button = widgets.Button(
            description='Remove Tenor', tooltip='Delete vol surface for this tenor')
        self.dropdown = widgets.Combobox(
            description='Tenor:', placeholder='Choose Tenor', options=[], 
            ensure_option=False)
        # self.dropdown.layout.min_width='auto'
        self.tenor = None

        self.selector = widgets.HBox(children=[self.add_button, self.del_button, self.dropdown])
        self.selector.layout.visibility='hidden'
        
        self.add_button.on_click(self.add_button_clicked)
        self.del_button.on_click(self.del_button_clicked)
        self.dropdown.observe(self.change_selection, 'value')
        self.obj = None
        
        super().__init__(
            children=[
                self.description,
                widgets.VBox(children=[self.selector, self.plot, self.data])
            ]
        )

    def change_selection(self, change):
        if change.new in self.obj:
            matrix = self.obj[change.new]
            self.data.value = to_json(matrix)
            self.update_plot(matrix)
            self.tenor = self.dropdown.value
            self.plot_label.text = 'Tenor: {}'.format(self.tenor)
            self.dropdown.unobserve(self.change_selection, 'value')
            self.dropdown.value=''
            self.dropdown.observe(self.change_selection, 'value')
    
    def add_button_clicked(self, b):
        val = self.dropdown.value
        if val:
            try:
                val = self.dropdown.value
                self.dropdown.options = tuple([str(x) for x in sorted(
                    [float(x) for x in (self.dropdown.options + (val,))])])
                self.tenor = val
                self.plot_label.text = 'Tenor: {}'.format(self.tenor)
                self.obj[val] = json.loads(riskflow_jupyter.rf.fields.default['Surface'])
                matrix = self.obj[val]
                self.data.value = to_json(matrix)
                self.update_plot(matrix)
            except:
                print('could not cast value to float')
            
        
    def del_button_clicked(self, b):
        val = self.dropdown.value if self.dropdown.value else self.tenor
        if val in self.dropdown.options:
            self.dropdown.options = tuple([x for x in self.dropdown.options if x!=val])
            print('inside - about to del', self.obj.keys())
            del self.obj[val]
            print('inside - about to del', self.obj.keys())
            if self.dropdown.options:
                self.tenor = self.dropdown.options[0]
                matrix = self.obj[self.tenor]
            else:
                self.tenor = ''
                matrix = [[]]
                
            self.dropdown.value = self.tenor
            print('inside - after del', self.obj.keys())
            self.data.value = to_json(matrix)
            self.update_plot(matrix)
        print("del Button clicked.", val)

    def observe(self, handler, prop, type='change'):

        def make_plot_fn():
            def update_plot(change):
                # call the original handler first
                handler(change)
                # now update the plot with the new data
                table_obj = json.loads(change.new)
                # store the result if this is a 3d obj
                if self.tenor is not None:
                    self.obj[self.tenor]=table_obj    
                # check if we have a valid table obj
                if np.all([len(x)>1 for x in table_obj[1:]]):
                    self.update_plot(table_obj)

            return update_plot

        if self.children:
            # link the observable function to the table widget (self.data)
            self.data.observe(make_plot_fn(), prop, type)
        else:
            super().observe(handler, prop, type)

    @staticmethod
    def make_faces_vectorized1(Nr, Nc):

        out = np.empty((Nr - 1, Nc - 1, 2, 3), dtype=int)

        r = np.arange(Nr * Nc).reshape(Nr, Nc)

        out[:, :, 0, 0] = r[:-1, :-1]
        out[:, :, 1, 0] = r[:-1, 1:]
        out[:, :, 0, 1] = r[:-1, 1:]

        out[:, :, 1, 1] = r[1:, 1:]
        out[:, :, :, 2] = r[1:, :-1, None]

        out.shape = (-1, 3)
        return out

    @staticmethod
    def interpolate_surface(json_list):
        moneyness = json_list[0][1:]
        e = []
        for p in json_list[1:]:
            e.extend([[m, p[0], v] for m, v in zip(moneyness, p[1:]) if v is not None])

        surface = np.array(e)
        expiry = [p[0] for p in json_list[1:]]
        return expiry, moneyness, np.array([np.interp(
            moneyness, surface[surface[:, 1] == x][:, 0], surface[surface[:, 1] == x][:, 2]) for x in expiry])

    def update_plot(self, json_list):
        e, moneyness, vol = Three.interpolate_surface(json_list)
        
        if min(moneyness)>0:
            scale = 2 / np.log(2)
            m = scale * np.log(moneyness)            
        else:
            m = np.array(moneyness)*100.0
            self.plot.axes=['moneyness (bps)', 'expiry','vol(\%)']
            
        v = vol * 100

        U, V = np.meshgrid(m, e)
        vertices = np.dstack([U, V, v]).astype(np.float32).reshape(-1, 3)
        indices = Three.make_faces_vectorized1(*v.shape).astype(np.uint32)
        if self.mesh is None:
            self.mesh = k3d.mesh(
                vertices, indices, flat_shading=False, attribute=v,
                side='double', color_map=k3d.basic_color_maps.Reds, color_range=[v.min(), v.max()]
            )
            self.plot_label = k3d.text2d('', position=(0, 0))
            # add the plots
            self.plot += self.mesh
            self.plot += self.plot_label
        else:
            self.mesh.attribute = v
            self.mesh.color_range = [v.min(), v.max()]
            self.mesh.vertices = vertices
            self.mesh.indices = indices

    @property
    def value(self):
        return to_json(self.obj)

    @value.setter
    def value(self, json_string):
        as_python = json.loads(json_string)
        self.obj = as_python
        if isinstance(as_python, dict):
            self.selector.layout.visibility='visible'
            self.dropdown.options = tuple(as_python.keys())
            selection = self.dropdown.options[0]
            self.dropdown.value = selection
            matrix = as_python[selection]
            data_value  = to_json(matrix)
        else:
            self.selector.layout.visibility='hidden'
            data_value = json_string
            matrix     = as_python
            
        self.data.value = data_value
        self.update_plot(matrix)

In [ ]:
riskflow_jupyter.rf.fields.default

In [ ]:
j=load_table_from_vol(vol)
#to_json(j)
#np.isnan(None)

In [ ]:
# g = np.vstack([x[0], heights]).tolist()
t = Three(description='Test')
t.observe(lambda x:x['new'], 'value')
t.value = to_json(j)

In [ ]:
t

In [ ]:
t.data.value


In [ ]:
data

In [ ]:
data.value

In [ ]:
plt_text1.visible=False

In [ ]:
t.dropdown.layout.max_width='250px'

In [ ]:
# t.selector.layout.visibility = 'hidden'
t.value

In [ ]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/vretiel/miniconda3/')
display(fc)

In [ ]:
fc.selected_filename

In [ ]:
cx

In [ ]:
pp = riskflow_jupyter.PortfolioPage(cx.current_cfg)

In [ ]:
pp.main_container

In [ ]:
rfp = riskflow_jupyter.RiskFactorsPage(cx.current_cfg)

In [ ]:
#rfp.main_container


In [ ]:
rfp.tree.checked


In [ ]:
a='InterestYieldVol.CAD_SMILE_ICE'

c= getattr(riskflow_jupyter.rf.riskfactors, riskflow_jupyter.rf.utils.check_rate_name(a)[0])

In [ ]:
c

In [ ]:
riskflow_jupyter.CalculationPage(cx.current_cfg)

In [ ]:
%debug